# Microscope sync device based on a 32-bit ARM microcontroller

In [1]:
from sync_dev import SyncDevice, props
import time

## Communication logging
You can log all serial port communication by setting the `log_file` argument when you connect to the sync device.

In [2]:
log_file = None                   # no logging
log_file = 'sync_device log.txt'  # save to file
log_file = 'print'                # print here


In [3]:
sd = SyncDevice("COM4", log_file=log_file)

2024-10-28 16:29:15.519 RX: Sync device is ready. Firmware version: 2.1.0

2024-10-28 16:29:15.549 TX: bytearray(b'GET\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')
2024-10-28 16:29:15.555 RX: 2.1.0

2024-10-28 16:29:15.555 TX: bytearray(b'FUN\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')


Let's disable logging, otherwise it will be too much text here

In [4]:
sd = SyncDevice("COM4", log_file=None)

COM4 is in use. Closing and re-opening...


You can get an overview of the system status with `get_status()` method, or by just printing the synchronization device:

In [5]:
sd

SYNC DEVICE v2.1.0
-- SYSTEM STATUS --
Event queue size: 0
System counter is RUNNING
System time: 0.217175 s

## Device properties
The sync device has properties (see *globals.h* file), which could be access using `get_property()` method or directly as a class attribute. Most of 
them are read-only, while others are read/write.

In [6]:
[p for p in dir(props) if not p.startswith("__")]

['ro_N_EVENTS',
 'ro_SYS_TIMER_OVF_COUNT',
 'ro_SYS_TIMER_PRESCALER',
 'ro_SYS_TIMER_STATUS',
 'ro_SYS_TIMER_VALUE',
 'ro_SYS_TIME_s',
 'ro_VERSION',
 'ro_WATCHDOG_TIMEOUT_ms',
 'rw_DFLT_PULSE_DURATION_us',
 'rw_INTLCK_ENABLED']

In [7]:
print(sd.get_property(props.ro_VERSION))

# or easier
print(sd.version)

2.1.0
2.1.0


In [8]:
# Is interlock active?
print(f"Interlock active? {sd.interlock_enabled}")

# Let's disable the interlock
sd.interlock_enabled = False

print(f"Interlock active? {sd.interlock_enabled}")

Interlock active? True
Interlock active? False


In [9]:
# You can't set a read-only property
sd.version = "x.y.z"

AttributeError: property 'version' of 'SyncDevice' object has no setter

## Creating events and sending signals
You can schedule events, such as set/reset pin, toggle pin, send pulse, etc. Each event has:
 * up to two arguments
 * a corresponding timestamp,
 * number of occurences,
 * and an interval between occurences

The microcontroller puts all events into an ordered priority queue, where they are sorted by timestamp, from the earliest to the latest. If the system timer is running, it will grab the next scheduled event and execute it exactly at the timestamp of this event. If necessary, the event will be rescheduled.

Let's schedule a couple of positive pulses ("PPL" command in the protocol). You will see signals on Arduino pins A0 and A1, as well as a list of events (method `get_events()`). The timestamp `ts` is relative to the current time point.

In [10]:
sd.pos_pulse("A1", 50000, ts=1000000, N=25, interval=70000)
sd.pos_pulse("A0", 8000, N=120, interval=50000)
sd.get_events()

[SET_PIN(16 , 0  ) at t=       4378ms. Call    120 times every         50 ms,
 SET_PIN(16 , 1  ) at t=       4420ms. Call    119 times every         50 ms,
 SET_PIN(24 , 1  ) at t=       5368ms. Call     25 times every         70 ms,
 SET_PIN(24 , 0  ) at t=       5418ms. Call     25 times every         70 ms]

### Context manager
In the previous example, events were scheduled using individual lines of Python code, which resulted in individual data transmission to the sync device. The instructions are processed immediately, however, they can arrive 10-25 ms apart from each other, and there is no way to predict the time delay your computer puts between them.

If you want tighter timings between the requested events, use the context manager. Every command within the context manager gets queued in a buffer, and the entire buffer is transmitted the moment you exit the context manager. The time delay between subsequent commands should be exactly 1.88ms (time to transmit 24 bytes at 115200 baud). Below is the example

In [11]:
with sd as dev:
    dev.pos_pulse("A12",100000, N=10, interval=500000, ts=0)
    dev.pos_pulse("A0", 100000, N=10, interval=500000, ts=5000)
    dev.pos_pulse("A12",100000, N=10, interval=500000, ts=10000)
    dev.pos_pulse("A1", 100000, N=10, interval=500000, ts=105000)
    dev.pos_pulse("A12",100000, N=10, interval=500000, ts=200000)
    dev.pos_pulse("A2", 100000, N=10, interval=500000, ts=205000)
    dev.pos_pulse("A12",100000, N=10, interval=500000, ts=300000)
    dev.pos_pulse("A3", 100000, N=10, interval=500000, ts=305000)
sd.get_events()

[SET_PIN(47 , 0  ) at t=      11374ms. Call     10 times every        500 ms,
 SET_PIN(16 , 0  ) at t=      11381ms. Call     10 times every        500 ms,
 SET_PIN(24 , 1  ) at t=      11385ms. Call     10 times every        500 ms,
 SET_PIN(47 , 0  ) at t=      11388ms. Call     10 times every        500 ms,
 SET_PIN(47 , 1  ) at t=      11482ms. Call     10 times every        500 ms,
 SET_PIN(24 , 0  ) at t=      11485ms. Call     10 times every        500 ms,
 SET_PIN(23 , 1  ) at t=      11489ms. Call     10 times every        500 ms,
 SET_PIN(47 , 0  ) at t=      11582ms. Call     10 times every        500 ms,
 SET_PIN(47 , 1  ) at t=      11586ms. Call     10 times every        500 ms,
 SET_PIN(23 , 0  ) at t=      11589ms. Call     10 times every        500 ms,
 SET_PIN(22 , 1  ) at t=      11593ms. Call     10 times every        500 ms,
 SET_PIN(47 , 0  ) at t=      11686ms. Call     10 times every        500 ms,
 SET_PIN(22 , 0  ) at t=      11693ms. Call     10 times every  

## Precise timings with stopping/starting system timer
If you have several events and need exact time intervals between them, you should stop system timer, program your events, and then re-start the system timer. No events are processes when the timer is stopped.


In [12]:
sd.running

True

In [13]:
sd.stop()
sd.running

False

In [14]:
sd.pos_pulse("A12",100000, N=10, interval=500000, ts=0)
sd.pos_pulse("A0", 100000, N=10, interval=500000, ts=5000)
sd.pos_pulse("A12",100000, N=10, interval=500000, ts=10000)
sd.pos_pulse("A1", 100000, N=10, interval=500000, ts=105000)
sd.pos_pulse("A12",100000, N=10, interval=500000, ts=200000)
sd.pos_pulse("A2", 100000, N=10, interval=500000, ts=205000)
sd.pos_pulse("A12",100000, N=10, interval=500000, ts=300000)
sd.pos_pulse("A3", 100000, N=10, interval=500000, ts=305000)
sd.get_events()

[SET_PIN(47 , 1  ) at t=          0ms. Call     10 times every        500 ms,
 SET_PIN(16 , 1  ) at t=          5ms. Call     10 times every        500 ms,
 SET_PIN(47 , 1  ) at t=         10ms. Call     10 times every        500 ms,
 SET_PIN(47 , 0  ) at t=        100ms. Call     10 times every        500 ms,
 SET_PIN(24 , 1  ) at t=        105ms. Call     10 times every        500 ms,
 SET_PIN(16 , 0  ) at t=        105ms. Call     10 times every        500 ms,
 SET_PIN(47 , 0  ) at t=        110ms. Call     10 times every        500 ms,
 SET_PIN(47 , 1  ) at t=        200ms. Call     10 times every        500 ms,
 SET_PIN(23 , 1  ) at t=        205ms. Call     10 times every        500 ms,
 SET_PIN(24 , 0  ) at t=        205ms. Call     10 times every        500 ms,
 SET_PIN(47 , 1  ) at t=        300ms. Call     10 times every        500 ms,
 SET_PIN(47 , 0  ) at t=        300ms. Call     10 times every        500 ms,
 SET_PIN(22 , 1  ) at t=        305ms. Call     10 times every  

Let's add two more events that would toggle pin D4 at close frequency, causing the connected LED to gradually change apparent brightness.

In [15]:
sd.tgl_pin("D17", N=10000, interval=500)
sd.tgl_pin("D17", N=10000, interval=501)
sd.get_events("us")

[SET_PIN(47 , 1  ) at t=          0us. Call     10 times every     500000 us,
 TGL_PIN(12 , 0  ) at t=          0us. Call  10000 times every        499 us,
 TGL_PIN(12 , 0  ) at t=          0us. Call  10000 times every        500 us,
 SET_PIN(16 , 1  ) at t=       5000us. Call     10 times every     500000 us,
 SET_PIN(47 , 1  ) at t=      10000us. Call     10 times every     500000 us,
 SET_PIN(47 , 0  ) at t=     100000us. Call     10 times every     500000 us,
 SET_PIN(24 , 1  ) at t=     105000us. Call     10 times every     500000 us,
 SET_PIN(16 , 0  ) at t=     105000us. Call     10 times every     500000 us,
 SET_PIN(47 , 0  ) at t=     110000us. Call     10 times every     500000 us,
 SET_PIN(47 , 1  ) at t=     200000us. Call     10 times every     500000 us,
 SET_PIN(23 , 1  ) at t=     205000us. Call     10 times every     500000 us,
 SET_PIN(24 , 0  ) at t=     205000us. Call     10 times every     500000 us,
 SET_PIN(47 , 1  ) at t=     300000us. Call     10 times every  

In [16]:
sd

SYNC DEVICE v2.1.0
-- SYSTEM STATUS --
Event queue size: 18
System counter is STOPPED
System time: 18.184052 s

In [17]:
# Now we have 18 scheduled events, let them run
sd.go()
time.sleep(1)
sd.get_events("us")

[TGL_PIN(12 , 0  ) at t=    1007916us. Call   7988 times every        500 us,
 TGL_PIN(12 , 0  ) at t=    1008115us. Call   7983 times every        499 us,
 SET_PIN(47 , 1  ) at t=    1010000us. Call      8 times every     500000 us,
 SET_PIN(47 , 0  ) at t=    1100000us. Call      8 times every     500000 us,
 SET_PIN(24 , 1  ) at t=    1105000us. Call      8 times every     500000 us,
 SET_PIN(16 , 0  ) at t=    1105000us. Call      8 times every     500000 us,
 SET_PIN(47 , 0  ) at t=    1110000us. Call      8 times every     500000 us,
 SET_PIN(47 , 1  ) at t=    1200000us. Call      8 times every     500000 us,
 SET_PIN(24 , 0  ) at t=    1205000us. Call      8 times every     500000 us,
 SET_PIN(23 , 1  ) at t=    1205000us. Call      8 times every     500000 us,
 SET_PIN(47 , 1  ) at t=    1300000us. Call      8 times every     500000 us,
 SET_PIN(47 , 0  ) at t=    1300000us. Call      8 times every     500000 us,
 SET_PIN(22 , 1  ) at t=    1305000us. Call      8 times every  

If you connect oscilloscope, you will see that the time intervals between the events are exactly what you requested, down to microseconds. You will see jitter is two or more events occur very close to each other (closer than ~10us). If two events are scheduled to happen at the same timestamp, their order is undefined. Avoid scheduling of same events if you can.

In [18]:
sd.stop()
for i in range(400):
    sd.tgl_pin("D17", ts=25*i, N=10000, interval=1000)
sd.get_events()

[TGL_PIN(12 , 0  ) at t=          0ms. Call  10000 times every          1 ms,
 TGL_PIN(12 , 0  ) at t=          0ms. Call  10000 times every          1 ms,
 TGL_PIN(12 , 0  ) at t=          0ms. Call  10000 times every          1 ms,
 TGL_PIN(12 , 0  ) at t=          0ms. Call  10000 times every          1 ms,
 TGL_PIN(12 , 0  ) at t=          0ms. Call  10000 times every          1 ms,
 TGL_PIN(12 , 0  ) at t=          0ms. Call  10000 times every          1 ms,
 TGL_PIN(12 , 0  ) at t=          0ms. Call  10000 times every          1 ms,
 TGL_PIN(12 , 0  ) at t=          0ms. Call  10000 times every          1 ms,
 TGL_PIN(12 , 0  ) at t=          0ms. Call  10000 times every          1 ms,
 TGL_PIN(12 , 0  ) at t=          0ms. Call  10000 times every          1 ms,
 TGL_PIN(12 , 0  ) at t=          0ms. Call  10000 times every          1 ms,
 TGL_PIN(12 , 0  ) at t=          0ms. Call  10000 times every          1 ms,
 TGL_PIN(12 , 0  ) at t=          0ms. Call  10000 times every  

Now we have too many events in the table. Once we start the microcontroller, it will slowly get overwhelmed and will start missing events. When this happens, it will automatically reset. It will send an error message to the host.

In [19]:
sd.go()
time.sleep(1)
print(sd.com.readall())

b'ERR - watchdog timeout, restarting system!\n\x00Sync device is ready. Firmware version: 2.1.0\n'


## Enabling/disabling pins
Each pin can be enabled or disabled using the "ENP" and "DSP" commands. A disabled pin still preserves internal state, but it lacks the electrical output.

In [20]:
with sd as dev:
    dev.tgl_pin("D17", N=5000, interval=1000)
    dev.tgl_pin("D17", N=5000, interval=1001)
    dev.tgl_pin("A0", N=5000, interval=1000)
    dev.tgl_pin("A0", N=5000, interval=1001)
time.sleep(0.5)
sd.disable_pin("D17")
time.sleep(0.5)
sd.enable_pin("D17")

## Interlock
Some pins (see *globals.h*) are assigned to laser shutters. These are additionally controlled by an interlock. The interlock periodically sends a short heart beat pulse on pin D13 and expects to receive this pulse on pin D12. If the pulse does not arrive, the laser shutter pins are immediately set to TTL low. The pins return to their normal function as soon as interlock detects pulse on D12. This mechanism does not affect the enable/disable status of the pins or their internal level, it's a separate layer of logic.

Sending hearbeat pulse through the interlock circuit makes the interlock highly reliable, as it detects not only the open circuit, but also short circuit to ground or TTL high.

If you don't need interlock, you can disable it until the next microcontroller reset.

In [21]:
sd.interlock_enabled

True

In [22]:
# A0 is disabled if interlock circuit is disrupted
sd.interlock_enabled = True
sd.tgl_pin("A0", N=10000, interval=1000)
sd.tgl_pin("A0", N=10000, interval=1001)
sd.tgl_pin("A12", N=10000, interval=1000)
sd.tgl_pin("A12", N=10000, interval=1001)

In [23]:
# Interlock is disabled, A0 is unaffected
sd.interlock_enabled = False
sd.tgl_pin("A0", N=10000, interval=1000)
sd.tgl_pin("A0", N=10000, interval=1001)
sd.tgl_pin("A12", N=10000, interval=1000)
sd.tgl_pin("A12", N=10000, interval=1001)